### Bayesian Statistics

Bayes theorem is used to update probability distributions after obtaining new data. Given two events, $A$ and $B$, the conditional probability if $A$ given $B$ is given by:

\begin{align}
    P(A | B) = \frac{P(B | A) P(A)}{P(B)}
\end{align}